In [1]:
# --- Cell 1: Imports and Setup ---
import pandas as pd
import numpy as np
import os
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns

# --- Import functions from your process_data.py file ---
try:
    from process_data import (
        load_battery_data,
        segment_discharge_cycles,
        filter_invalid_cycles,
        extract_cycle_features, # Includes core and new statistical features
        calculate_q_initial_and_soh,
        add_health_indicators,
        process_battery_dataset, # Simplified orchestrator
        DATASET_ROOT_DIRECTORY,
        FOLDERS_TO_PROCESS,
        NOMINAL_CAPACITY_AH
    )
    print("Successfully imported functions from process_data.py (simplified version)")
except ImportError as e:
    print(f"Error importing from process_data.py: {e}")
    # Add sys.path.append if needed

# --- Configuration for this Notebook Run ---
SINGLE_BATTERY_TO_PROCESS = None # Or None for all

TARGET_BATTERY_SUBFOLDER = None
if SINGLE_BATTERY_TO_PROCESS:
    for folder in FOLDERS_TO_PROCESS:
        potential_path = os.path.join(DATASET_ROOT_DIRECTORY, folder, f"{SINGLE_BATTERY_TO_PROCESS}.csv")
        if os.path.exists(potential_path):
            TARGET_BATTERY_SUBFOLDER = folder
            print(f"Target battery '{SINGLE_BATTERY_TO_PROCESS}' found in: '{TARGET_BATTERY_SUBFOLDER}'")
            break
    if not TARGET_BATTERY_SUBFOLDER and SINGLE_BATTERY_TO_PROCESS: # Check if still None after loop AND single_batt was set
        print(f"WARNING: Target '{SINGLE_BATTERY_TO_PROCESS}.csv' not found in specified folders.")

NOTEBOOK_DATASET_ROOT = DATASET_ROOT_DIRECTORY
print(f"Dataset root (from process_data.py): {NOTEBOOK_DATASET_ROOT}")
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

Successfully imported functions from process_data.py (simplified version)
Dataset root (from process_data.py): battery_alt_dataset


In [2]:
# --- Cell 2: Detailed Test of extract_cycle_features for ONE cycle ---
if SINGLE_BATTERY_TO_PROCESS and TARGET_BATTERY_SUBFOLDER:
    test_battery_id = SINGLE_BATTERY_TO_PROCESS
    test_file_path = os.path.join(NOTEBOOK_DATASET_ROOT, TARGET_BATTERY_SUBFOLDER, f"{test_battery_id}.csv")
    print(f"--- Testing extract_cycle_features for battery: {test_battery_id} ---")
    raw_df_single = load_battery_data(test_file_path)
    if raw_df_single is not None and not raw_df_single.empty:
        segmented_cycles_single = segment_discharge_cycles(raw_df_single, test_battery_id)
        if segmented_cycles_single:
            valid_cycles_single = filter_invalid_cycles(segmented_cycles_single, test_battery_id)
            if valid_cycles_single:
                cycle_to_test_info = valid_cycles_single[0]
                print(f"\nTesting feature extraction for cycle: {cycle_to_test_info['cycle_number']}")
                extracted_features_dict = extract_cycle_features(cycle_to_test_info)
                print("\n--- Extracted Features for Test Cycle (Simplified) ---")
                for key, value in extracted_features_dict.items():
                    print(f"  {key}: {value}")
            else: print("No valid cycles for test battery.")
        else: print("No segmented cycles for test battery.")
    else: print(f"Failed to load raw data for {test_battery_id}.")
elif SINGLE_BATTERY_TO_PROCESS:
    print(f"Cannot run detailed test for '{SINGLE_BATTERY_TO_PROCESS}', subfolder not identified or load failed.")
else:
    print("Skipping detailed test (SINGLE_BATTERY_TO_PROCESS not set).")

Skipping detailed test (SINGLE_BATTERY_TO_PROCESS not set).


In [3]:
# --- Cell 3: Process Dataset (Simplified Features) ---
print(f"\n--- Starting Main Dataset Processing (Simplified Features) ---")
if SINGLE_BATTERY_TO_PROCESS:
    print(f"Mode: Processing SINGLE battery ID: '{SINGLE_BATTERY_TO_PROCESS}'")
else:
    print(f"Mode: Processing ALL batteries in folders: {FOLDERS_TO_PROCESS}")

# Call the simplified main processing function
master_df_simplified = process_battery_dataset(
    root_dir=DATASET_ROOT_DIRECTORY,
    folders_list=FOLDERS_TO_PROCESS,
    single_battery_id=SINGLE_BATTERY_TO_PROCESS
)

# --- Diagnostics for Cell 3 (Simplified) ---
if not master_df_simplified.empty:
    print("\n--- Master DataFrame Diagnostics (Simplified Features) ---")
    print(f"Master DataFrame shape: {master_df_simplified.shape}")
    print(f"Unique batteries processed: {master_df_simplified['battery_id'].nunique()}")
    print("\nMaster DataFrame info():")
    master_df_simplified.info(verbose=True, show_counts=True)
    print("\nMaster DataFrame head:")
    display(master_df_simplified.head(10))

    # Check for some of the new statistical features
    new_stat_features_to_check = [
        'voltage_std_V', 'voltage_skewness', 'voltage_kurtosis', 'voltage_p50_V',
        'current_std_A', 'current_skewness', 'current_kurtosis', 'current_p50_A',
        'temp_std_C', 'temp_skewness', 'temp_kurtosis', 'temp_p50_C'
    ]
    print("\nChecking new statistical features:")
    for feat in new_stat_features_to_check:
        if feat in master_df_simplified.columns:
            non_nan = master_df_simplified[feat].notna().sum()
            print(f"  Feature '{feat}': Present (Non-NaN: {non_nan} / {len(master_df_simplified)})")
        else:
            print(f"  Feature '{feat}': MISSING")
else:
    print("\nMaster DataFrame (Simplified) is empty. No data processed or error.")


--- Starting Main Dataset Processing (Simplified Features) ---
Mode: Processing ALL batteries in folders: ['regular_alt_batteries', 'recommissioned_batteries']
Scanning folder: battery_alt_dataset/regular_alt_batteries

Processing battery: battery00 from file battery00.csv


/Users/recepfurkanakin/PycharmProjects/BatteryDatasetEDA/process_data.py:69: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_out[f'{col}_roll_mean_{w}'] = df_out[col].rolling(window=w, min_periods=1).mean()
/Users/recepfurkanakin/PycharmProjects/BatteryDatasetEDA/process_data.py:71: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_out[f'{col}_roll_std_{w}'] = df_out[col].rolling(window=w, min_periods=1).std()
/Users/recepfurkanakin/PycharmProjects/BatteryDatasetEDA/process_data.py:73: PerformanceWarning: DataFrame is highly fra


Processing battery: battery01 from file battery01.csv


/Users/recepfurkanakin/PycharmProjects/BatteryDatasetEDA/process_data.py:69: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_out[f'{col}_roll_mean_{w}'] = df_out[col].rolling(window=w, min_periods=1).mean()
/Users/recepfurkanakin/PycharmProjects/BatteryDatasetEDA/process_data.py:71: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_out[f'{col}_roll_std_{w}'] = df_out[col].rolling(window=w, min_periods=1).std()
/Users/recepfurkanakin/PycharmProjects/BatteryDatasetEDA/process_data.py:73: PerformanceWarning: DataFrame is highly fra


Processing battery: battery10 from file battery10.csv


...

/Users/recepfurkanakin/PycharmProjects/BatteryDatasetEDA/process_data.py:69: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_out[f'{col}_roll_mean_{w}'] = df_out[col].rolling(window=w, min_periods=1).mean()
/Users/recepfurkanakin/PycharmProjects/BatteryDatasetEDA/process_data.py:71: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_out[f'{col}_roll_std_{w}'] = df_out[col].rolling(window=w, min_periods=1).std()
/Users/recepfurkanakin/PycharmProjects/BatteryDatasetEDA/process_data.py:73: PerformanceWarning: DataFrame is highly fra


Processing battery: battery53 from file battery53.csv

Concatenating data from all processed batteries...
Feature extraction and SOH calculation complete.

--- Master DataFrame Diagnostics (Simplified Features) ---
Master DataFrame shape: (8220, 200)
Unique batteries processed: 23

Master DataFrame info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8220 entries, 0 to 8219
Data columns (total 200 columns):
 #    Column                                Non-Null Count  Dtype  
---   ------                                --------------  -----  
 0    battery_id                            8220 non-null   object 
 1    cycle_number                          8220 non-null   int64  
 2    is_reference_cycle                    8220 non-null   bool   
 3    discharge_duration_s                  8220 non-null   float64
 4    capacity_Ah                           8220 non-null   float64
 5    energy_Wh                             8220 non-null   float64
 6    avg_current_A                    

/Users/recepfurkanakin/PycharmProjects/BatteryDatasetEDA/process_data.py:69: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_out[f'{col}_roll_mean_{w}'] = df_out[col].rolling(window=w, min_periods=1).mean()
/Users/recepfurkanakin/PycharmProjects/BatteryDatasetEDA/process_data.py:71: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_out[f'{col}_roll_std_{w}'] = df_out[col].rolling(window=w, min_periods=1).std()
/Users/recepfurkanakin/PycharmProjects/BatteryDatasetEDA/process_data.py:73: PerformanceWarning: DataFrame is highly fra

,battery_id,cycle_number,is_reference_cycle,discharge_duration_s,capacity_Ah,energy_Wh,avg_current_A,avg_voltage_V,start_voltage_V,end_voltage_V,delta_voltage_V,avg_power_W,avg_temp_C,start_temp_C,end_temp_C,delta_temp_C,max_temp_C,internal_resistance_ohm,voltage_std_V,voltage_variance_V2,voltage_skewness,voltage_kurtosis,voltage_p10_V,voltage_p25_V,voltage_p50_V,voltage_p75_V,voltage_p90_V,current_std_A,current_variance_A2,current_skewness,current_kurtosis,current_p10_A,current_p25_A,current_p50_A,current_p75_A,current_p90_A,temp_std_C,temp_variance_C2,temp_skewness,temp_kurtosis,temp_p10_C,temp_p25_C,temp_p50_C,temp_p75_C,temp_p90_C,dVdQ_mean_V_mAh,dVdQ_std_V_mAh,dVdQ_min_V_mAh,dVdQ_max_V_mAh,dVdQ_skewness,...,current_skewness_roll_std_5,current_skewness_diff_5,current_skewness_roll_mean_10,current_skewness_roll_std_10,current_skewness_diff_10,current_kurtosis_roll_mean_3,current_kurtosis_roll_std_3,current_kurtosis_diff_3,current_kurtosis_roll_mean_5,current_kurtosis_roll_std_5,current_kurtosis_diff_5,current_kurtosis_roll_mean_10,current_kurtosis_roll_std_10,current_kurtosis_diff_10,temp_skewness_roll_mean_3,temp_skewness_roll_std_3,temp_skewness_diff_3,temp_skewness_roll_mean_5,temp_skewness_roll_std_5,temp_skewness_diff_5,temp_skewness_roll_mean_10,temp_skewness_roll_std_10,temp_skewness_diff_10,temp_kurtosis_roll_mean_3,temp_kurtosis_roll_std_3,temp_kurtosis_diff_3,temp_kurtosis_roll_mean_5,temp_kurtosis_roll_std_5,temp_kurtosis_diff_5,temp_kurtosis_roll_mean_10,temp_kurtosis_roll_std_10,temp_kurtosis_diff_10,dVdQ_mean_V_mAh_roll_mean_3,dVdQ_mean_V_mAh_roll_std_3,dVdQ_mean_V_mAh_diff_3,dVdQ_mean_V_mAh_roll_mean_5,dVdQ_mean_V_mAh_roll_std_5,dVdQ_mean_V_mAh_diff_5,dVdQ_mean_V_mAh_roll_mean_10,dVdQ_mean_V_mAh_roll_std_10,dVdQ_mean_V_mAh_diff_10,V_slope_seg2_V_s_roll_mean_3,V_slope_seg2_V_s_roll_std_3,V_slope_seg2_V_s_diff_3,V_slope_seg2_V_s_roll_mean_5,V_slope_seg2_V_s_roll_std_5,V_slope_seg2_V_s_diff_5,V_slope_seg2_V_s_roll_mean_10,V_slope_seg2_V_s_roll_std_10,V_slope_seg2_V_s_diff_10
0,battery00,0,True,3502.292,2.452221,17.799597,2.520635,7.259734,8.329,4.814,3.515,18.299208,25.917209,23.155,29.161,6.006,29.161,0.014277,0.583853,0.340884,-0.586329,0.448920,6.5610,6.85100,7.2810,7.74700,7.9890,0.000584,3.407039e-07,0.778159,2.500026,2.520000,2.520000,2.521000,2.521000,2.521000,1.053858,1.110617,-0.074045,0.197747,24.3624,25.35000,26.0400,26.53800,27.1790,-0.001445,0.003581,-0.039693,0.001521,-6.611756,...,NaN,NaN,0.778159,NaN,NaN,2.500026,NaN,NaN,2.500026,NaN,NaN,2.500026,NaN,NaN,-0.074045,NaN,NaN,-0.074045,NaN,NaN,-0.074045,NaN,NaN,0.197747,NaN,NaN,0.197747,NaN,NaN,0.197747,NaN,NaN,-0.001445,NaN,NaN,-0.001445,NaN,NaN,-0.001445,NaN,NaN,-0.000516,NaN,NaN,-0.000516,NaN,NaN,-0.000516,NaN,NaN
1,battery00,1,False,537.423,2.432716,15.685112,16.281695,6.451526,8.657,4.921,3.736,104.977693,64.833192,22.350,94.618,72.268,94.618,0.107324,0.485558,0.235767,-0.315447,0.235715,5.8378,6.10600,6.4720,6.88150,7.0310,0.662160,4.384563e-01,-23.621412,556.647479,16.286718,16.298038,16.307927,16.324139,16.336095,19.297919,372.409687,-0.366493,-0.974483,36.1252,49.91450,67.5130,82.33650,88.9124,-0.002052,0.025265,-0.598146,0.004871,-23.369586,...,17.253102,NaN,-11.421627,17.253102,NaN,279.573753,391.841422,NaN,279.573753,391.841422,NaN,279.573753,391.841422,NaN,-0.220269,0.206792,NaN,-0.220269,0.206792,NaN,-0.220269,0.206792,NaN,-0.388368,0.828891,NaN,-0.388368,0.828891,NaN,-0.388368,0.828891,NaN,-0.001748,0.000429,NaN,-0.001748,0.000429,NaN,-0.001748,0.000429,NaN,-0.001757,0.001755,NaN,-0.001757,0.001755,NaN,-0.001757,0.001755,NaN
2,battery00,2,False,539.307,2.440109,15.797369,16.274049,6.476988,8.703,4.914,3.789,105.341179,67.258282,26.872,96.305,69.433,96.305,0.100823,0.508305,0.258374,-0.233617,0.079900,5.8464,6.11350,6.4850,6.91175,7.1062,0.663641,4.404188e-01,-23.549647,553.389937,16.277280,16.287931,16.301396,16.319070,16.329650,18.547374,344.005092,-0.308959,-0.997889,40.4253,52.77700,69.2865,82.44900,91.7131,-0.002028,0.023328,-0.550


Checking new statistical features:
  Feature 'voltage_std_V': Present (Non-NaN: 8220 / 8220)
  Feature 'voltage_skewness': Present (Non-NaN: 8220 / 8220)
  Feature 'voltage_kurtosis': Present (Non-NaN: 8220 / 8220)
  Feature 'voltage_p50_V': Present (Non-NaN: 8220 / 8220)
  Feature 'current_std_A': Present (Non-NaN: 8220 / 8220)
  Feature 'current_skewness': Present (Non-NaN: 8220 / 8220)
  Feature 'current_kurtosis': Present (Non-NaN: 8220 / 8220)
  Feature 'current_p50_A': Present (Non-NaN: 8220 / 8220)
  Feature 'temp_std_C': Present (Non-NaN: 8220 / 8220)
  Feature 'temp_skewness': Present (Non-NaN: 8219 / 8220)
  Feature 'temp_kurtosis': Present (Non-NaN: 8219 / 8220)
  Feature 'temp_p50_C': Present (Non-NaN: 8220 / 8220)


In [4]:
# --- Cell 4: Save Simplified Processed DataFrame ---
if not master_df_simplified.empty:
    output_filename = f"New_Features_Added"
    if SINGLE_BATTERY_TO_PROCESS:
        sanitized_id = SINGLE_BATTERY_TO_PROCESS.replace('.', '_').replace('/', '_')
        output_filename += f"_{sanitized_id}"
    else:
        output_filename += "_ALL"
    output_filename += ".csv"
    try:
        master_df_simplified.to_csv(output_filename, index=False)
        print(f"\nSaved simplified data to: {output_filename} (Shape: {master_df_simplified.shape})")
    except Exception as e:
        print(f"\nError saving simplified DataFrame: {e}")
else:
    print("\nMaster DataFrame (Simplified) is empty. Nothing to save.")


Saved simplified data to: New_Features_Added_ALL.csv (Shape: (8220, 200))


In [13]:
# Master_df_simplified columns
print("\n--- Master DataFrame (Simplified) Columns ---")
print(master_df_simplified.columns.tolist())


--- Master DataFrame (Simplified) Columns ---
['battery_id', 'cycle_number', 'is_reference_cycle', 'discharge_duration_s', 'capacity_Ah', 'energy_Wh', 'avg_current_A', 'avg_voltage_V', 'start_voltage_V', 'end_voltage_V', 'delta_voltage_V', 'avg_power_W', 'avg_temp_C', 'start_temp_C', 'end_temp_C', 'delta_temp_C', 'max_temp_C', 'internal_resistance_ohm', 'voltage_std_V', 'voltage_variance_V2', 'voltage_skewness', 'voltage_kurtosis', 'voltage_p10_V', 'voltage_p25_V', 'voltage_p50_V', 'voltage_p75_V', 'voltage_p90_V', 'current_std_A', 'current_variance_A2', 'current_skewness', 'current_kurtosis', 'current_p10_A', 'current_p25_A', 'current_p50_A', 'current_p75_A', 'current_p90_A', 'temp_std_C', 'temp_variance_C2', 'temp_skewness', 'temp_kurtosis', 'temp_p10_C', 'temp_p25_C', 'temp_p50_C', 'temp_p75_C', 'temp_p90_C', 'dVdQ_mean_V_mAh', 'dVdQ_std_V_mAh', 'dVdQ_min_V_mAh', 'dVdQ_max_V_mAh', 'dVdQ_skewness', 'dVdQ_kurtosis', 'V_slope_seg1_V_s', 'V_slope_seg2_V_s', 'V_slope_seg3_V_s', 'q_initi

In [12]:
import platform


platform.machine()

'arm64'